In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

### # quality check for coaches

In [0]:
expect_nocs ={
        "rule1":"country is not null",
        "rule2":"note is not null"
    }


In [0]:
expect_events ={
        "rule1":"event is not null",
        "rule2":"sport is not null"
    }

## Coaches DLT

In [0]:
@dlt.view

def view_coaches():
    df = spark.readStream.table("olympics_cata.silver.coaches")
    return df


In [0]:
import dlt

# define expectations
expect_coaches = {
    "non_null_id": "code IS NOT NULL",
    "valid_country": "country IS NOT NULL"
}

@dlt.table
@dlt.expect_all(expect_coaches)
def coaches():
    df = spark.readStream.table("LIVE.view_coaches")
    return df
 

## nocs DLT


In [0]:
@dlt.view

def view_nocs():
    df = spark.readStream.table("olympics_cata.silver.nocs")
    return df
    

In [0]:
@dlt.table
@dlt.expect_all_or_drop(expect_nocs)
def nocs():
    df = spark.readStream.table("LIVE.view_nocs")
    return df  

## Event DLT

In [0]:
@dlt.view

def view_events():
    df = spark.readStream.table("olympics_cata.silver.events")
    return df

In [0]:
@dlt.table
@dlt.expect_all(expect_events)
def events():
    df = spark.readStream.table("LIVE.view_events")
    return df  

### CDC on athletes table

In [0]:
@dlt.view

def view_athletes():
    df = spark.readStream.table("olympics_cata.silver.athletes")
    return df

In [0]:
dlt.create_streaming_table("athletes")

In [0]:
dlt.create_auto_cdc_flow(
  target = "athletes",
  source = "view_athletes",
  keys = ["code"],
  sequence_by = col("height"),
  stored_as_scd_type = 1
)